In [20]:
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, scale
import seaborn as sns
from pyearth import Earth
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pp = pprint.PrettyPrinter()

import TraitData
from imp import reload
from permutation_analysis import Permutation
import shap

In [2]:
SCORING = "neg_mean_squared_error"

# Model Interpretability Exploration

## Data Setup

In [65]:
reload(TraitData)
responseVar = "migration_m"

plants_master = pd.read_csv("../data/plants5.csv")


drop_features = ["Taxon",
                 "migr_sterr_m", 
                 "shift + 2SE", 
                 'signif_shift',
                 "signif_shift2",
                 "dispmode01",
                 "DispModeEng", ## what is this
                 "shift + 2SE",
                ]

categorical_features = ["oceanity",
                        "dispersal_mode",
                        "BreedSysCode",
                        "Grime"]

td = TraitData.TraitData("../data/plants5.csv",
               responseVar,
               drop_features,
               categorical_features,
               dropNA=1)

X, x_test, Y, y_test = td.train_test_split(0.30)

shapdata = shap.DenseData(X, td.feature_names)


## SVR setup:

In [66]:
from sklearn.svm import SVR


base = SVR()
errs = cross_val_score(base, scale(X), Y, cv=LeaveOneOut(), scoring=SCORING, n_jobs=-1)
-errs.mean()


26.238067371993722

## Model Interpretation

In [67]:
base.fit(scale(X), Y)
explainer = shap.KernelExplainer(base.predict, shapdata)

In [69]:
print(base.predict(scale(x_test)))
print(y_test)

[ 1.83179648  1.908906    3.53114339  1.47204416  1.41309844  2.76332877
  2.62748256  2.28146279  1.73424122  0.48188032  1.56503403  0.07473015
  0.57952706  1.52529478  2.2536919   2.00322577  2.34223888  0.6393797
  1.90656272  2.3086713   1.91226774  2.11956473  1.84955814  1.19981293
  0.69551212  2.30431944  0.56205254  0.83864775  0.91079038  2.53754479
  0.53736832  1.46943325  1.4202512   0.3947859   0.96250522  1.31558424
  1.06971854  2.82054004  2.28851518  2.47134681]
41     10.4
10      8.9
32      3.0
120     0.7
40      4.8
21      4.3
76      3.0
18      3.2
108     0.0
58      9.3
44      5.9
34      1.1
127     7.0
95     -5.8
75      0.0
59      3.5
125     2.7
82     -2.1
26      4.7
72     -4.2
117    -0.6
12      0.1
68      0.0
55      3.1
119     4.6
37     -1.4
69      0.6
126     0.0
7      -0.4
92      0.1
25     -5.2
128     6.1
62      0.1
11      9.9
112     2.2
47      3.7
85      1.6
38     -5.6
124     3.1
116     1.7
Name: migration_m, dtype: float64